In [11]:
import json
import pandas as pd
import numpy as np
import urllib2
import codecs
import requests
from tqdm import tnrange as prange, tqdm_notebook as pbar

In [62]:
data = pd.read_csv('data-01-structure-01.xls - data-01-structure-01.csv', sep=',', )
print len(data)
data.head(10)

65520


,OWNER,INN,KPP
0,"ЗАО ""ФЕСТИНА-РУС"" ...",7729405456,772901001
1,"ОАО ""Авиакомпания ""Сибирь"" ...",5448100656,544801001
2,"ЗАО ""Вестинтертранс"" ...",7733106829,773301001
3,"ООО ""Топаз"" ...",7825438239,782501001
4,"ОАО ""Шереметьево-Карго"" ...",7712014197,771201001
5,"ЗАО ""Авиационная компания ""ПОЛЕТ"" ...",3128000948,312801001
6,"ООО ""ТНТ Экспресс Уорлдуайт (СНГ)"" ...",7743003117,774301001
7,"ОАО ""Сахалинское морское пароходство"" ...",6509000854,650901001
8,"ООО ""ТРАНС-БИЗНЕС СЕВЕРО-ЗАПАД"" ...",7820019812,782001001
9,"ООО ""ВЕТТ ТРАНС"" ...",7716165694,507701001


In [93]:
def try_inc(dictionary, key, value):
    if key not in dictionary:
        dictionary[key] = value
    else:
        dictionary[key] += value

In [100]:
url = 'http://openapi.clearspending.ru/restapi/v3/suppliers/get/?'
name_to_count = {}
name_to_sum = {}
for own in pbar(data.values):
    this_inn = own[1]
    this_kpp = own[2]
    res_url = ''
    if this_inn:
        res_url = url + 'inn='+str(this_kpp)
    else:
        continue
    if len(res_url) == 0:
        continue
    page = requests.get(res_url, allow_redirects=False)
    if len(page.text) < 1000:
        continue
    parsed = json.loads(page.text)
    if len(own[0]) < 3:
        continue
    if 'contractsSum' not in parsed['suppliers']['data'][0]:
        continue
    try_inc(name_to_sum, own[0], parsed['suppliers']['data'][0]['contractsSum'])
    try_inc(name_to_count, own[0], parsed['suppliers']['data'][0]['contractsCount'])
print len(name_to_sum.items())

KeyboardInterrupt: 

In [102]:
ptr = pd.DataFrame(name_to_sum.items())
ptr.columns = ['name','sum']
ptr['count'] = name_to_count.values()
print len(ptr)
ptr.head()

228


,name,sum,count
0,"Общество с ограниченной ответственностью ""ВИТМ...",10261075.49,24
1,"Частное унитарное транспортное предприятие ""М-...",10261075.49,24
2,"Товарищество с ограниченной ответственностью ""...",10261075.49,24
3,"Общество с ограниченной ответственностью ""Мико...",20522150.98,48
4,"Общество с дополнительной ответственностью ""Бе...",10261075.49,24


In [103]:
ptr.sort_values(['count'], ascending=False).head(10)

,name,sum,count
192,...,5.294715e+09,12384
125,"Общество с ограниченной ответственностью ""БЕЛТ...",1.641772e+08,384
225,"Частное унитарное транспортное предприятие ""Тр...",1.436551e+08,336
157,"Общество с дополнительной ответственностью ""Ва...",1.436551e+08,336
113,Иностранное частное унитарное транспортно-эксп...,1.436551e+08,336
116,"Общество с ограниченной ответственностью ""ТНА""...",1.436551e+08,336
208,"Открытое акционерное общество ""Миноблавтотранс...",1.333940e+08,312
25,"Общество с ограниченной ответственностью ""ОРВЕ...",1.231329e+08,288
53,Иностранное частное транспортно-экспедиционное...,1.231329e+08,288
159,Иностранное частное унитарное транспортное пре...,1.128718e+08,264


In [104]:
ptr.to_csv('ptr.csv', index=False)